In [1]:
# A Convolutional Network implementation example using TensorFlow library.
# This example is using the MNIST database of handwritten digits
# (http://yann.lecun.com/exdb/mnist/)

# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
# import sys
# print(sys.path)
# sys.path.insert(1,'/tensorflow/tensorflow/g3doc/tutorials/mnist')

In [3]:
# import pip
# def install(package):
#     pip.main(['install', package])
# if __name__ == '__main__':
#     install('pip install --upgrade pip')

# --upgrade https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-0.8.0-cp27-none-linux_x86_64.whl
# pip install --upgrade pip

In [4]:
# Import MINST data

# import input_data
import tensorflow.examples.tutorials.mnist.input_data as input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
import tensorflow as tf

In [6]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 20

In [7]:
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

In [8]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [9]:
# Create model
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], 
                                                  padding='SAME'),b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, _dropout)

    # Convolution Layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    conv2 = tf.nn.dropout(conv2, _dropout)

    # Fully connected layer
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout) # Apply Dropout

    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

In [10]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), 
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), 
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])), 
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [11]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

In [12]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [13]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
# Initializing the variables
init = tf.initialize_all_variables()

In [15]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], 
                                                             y: mnist.test.labels[:256], 
                                                             keep_prob: 1.})

Iter 2560, Minibatch Loss= 14789.485352, Training Accuracy= 0.40625
Iter 5120, Minibatch Loss= 5940.838867, Training Accuracy= 0.59375
Iter 7680, Minibatch Loss= 7475.124512, Training Accuracy= 0.57031
Iter 10240, Minibatch Loss= 3612.242188, Training Accuracy= 0.74219
Iter 12800, Minibatch Loss= 4316.957520, Training Accuracy= 0.73438
Iter 15360, Minibatch Loss= 2164.175293, Training Accuracy= 0.81250
Iter 17920, Minibatch Loss= 2765.583496, Training Accuracy= 0.77344
Iter 20480, Minibatch Loss= 1218.909790, Training Accuracy= 0.91406
Iter 23040, Minibatch Loss= 1589.945312, Training Accuracy= 0.86719
Iter 25600, Minibatch Loss= 1814.135010, Training Accuracy= 0.85156
Iter 28160, Minibatch Loss= 2203.304688, Training Accuracy= 0.83594
Iter 30720, Minibatch Loss= 1303.687134, Training Accuracy= 0.86719
Iter 33280, Minibatch Loss= 1218.301025, Training Accuracy= 0.84375
Iter 35840, Minibatch Loss= 655.457031, Training Accuracy= 0.91406
Iter 38400, Minibatch Loss= 510.231140, Training Ac

In [17]:
import numpy as np

In [21]:
mnist.train.num_examples

55000

In [23]:
mnist.train.images

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [28]:
mnist.train.images.shape

(55000, 784)

In [29]:
weights

{'out': <tensorflow.python.ops.variables.Variable at 0x7f859c290310>,
 'wc1': <tensorflow.python.ops.variables.Variable at 0x7f85b60b0cd0>,
 'wc2': <tensorflow.python.ops.variables.Variable at 0x7f859c286e50>,
 'wd1': <tensorflow.python.ops.variables.Variable at 0x7f859c286490>}

In [57]:
with tf.Session() as sess:
    sess.run(init)
    print('wc1 shape is ',weights['wc1'].eval().shape)
    print('wc2 shape is ',weights['wc2'].eval().shape)
    print('wd1 shape is ',weights['wd1'].eval().shape)
    print('out shape is ',weights['out'].eval().shape)
    print('batch_xs shape is ',batch_xs.shape)
    print('batch_ys shape is ',batch_ys.shape)

('wc1 shape is ', (5, 5, 1, 32))
('wc2 shape is ', (5, 5, 32, 64))
('wd1 shape is ', (3136, 1024))
('out shape is ', (1024, 10))
('batch_xs shape is ', (128, 784))
('batch_ys shape is ', (128, 10))
